# 🧠 Optimizing Quick Commerce Logistics in Kochi (Google Colab Version)
This notebook runs fully in **Google Colab**, using real geospatial data from **OpenStreetMap (OSM)** and optimization models (**PuLP** and **OR-Tools**).

It replicates the MBA project *“Evaluating the Efficiency of Quick Commerce Logistics in Kochi and Its Impact on Customer Satisfaction.”*

➡️ **To run:** Just execute each cell below — it installs all dependencies automatically.

In [ ]:
# 📦 Install dependencies
!pip install osmnx geopandas pulp ortools folium matplotlib numpy pandas

In [ ]:
# 🗺️ Import libraries
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import pulp
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import folium, math
print('✅ All libraries loaded successfully.')

In [ ]:
# 🏙️ Fetch Kochi boundary and road network
koch = ox.geocode_to_gdf('Kochi, Kerala, India')
G = ox.graph_from_place('Kochi, Kerala, India', network_type='drive')
print('Kochi boundary and road network fetched!')
koch.plot()

In [ ]:
# 📊 Create sample demand zones and facilities
zones = gpd.GeoDataFrame({'zone_id': [f'Z{i}' for i in range(1,7)], 'demand': [200,150,180,130,160,140]})
facilities = gpd.GeoDataFrame({'fac_id': ['F1','F2','F3'], 'capacity':[500,600,400], 'fixed_cost':[1000,1200,900]})
transport_cost = np.random.randint(8,15,(3,6))
print('Sample data ready.')

In [ ]:
# ⚙️ Solve facility location using PuLP
prob = pulp.LpProblem('Facility_Location', pulp.LpMinimize)
open_var = pulp.LpVariable.dicts('open', range(3), 0, 1, cat='Binary')
flow = pulp.LpVariable.dicts('flow', [(i,j) for i in range(3) for j in range(6)], 0)
prob += pulp.lpSum(open_var[i]*facilities.fixed_cost[i] for i in range(3)) + pulp.lpSum(flow[(i,j)]*transport_cost[i,j] for i in range(3) for j in range(6))
for j in range(6): prob += pulp.lpSum(flow[(i,j)] for i in range(3)) == zones.demand[j]
for i in range(3): prob += pulp.lpSum(flow[(i,j)] for j in range(6)) <= facilities.capacity[i]*open_var[i]
prob.solve(pulp.PULP_CBC_CMD(msg=0))
print('Status:', pulp.LpStatus[prob.status])
print('Total Cost =', pulp.value(prob.objective))

In [ ]:
# 🚚 Vehicle Routing Problem (VRP) Demo using OR-Tools
distance_matrix = np.random.randint(2,20,(7,7))
manager = pywrapcp.RoutingIndexManager(7, 2, 0)
routing = pywrapcp.RoutingModel(manager)
def dist_callback(from_idx,to_idx):
 from_node,to_node = manager.IndexToNode(from_idx),manager.IndexToNode(to_idx)
 return distance_matrix[from_node][to_node]
transit_idx = routing.RegisterTransitCallback(dist_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_idx)
search_params = pywrapcp.DefaultRoutingSearchParameters()
search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
solution = routing.SolveWithParameters(search_params)
if solution:
 for v in range(2):
  idx = routing.Start(v)
  route=[]
  while not routing.IsEnd(idx):
   route.append(manager.IndexToNode(idx))
   idx = solution.Value(routing.NextVar(idx))
  print(f'Vehicle {v+1} route:', route)